In [1]:
import os
print(os.getcwd())
os.chdir('d:\\vscode_machineLearning\\internship\\Customer-Churn-Prediction')
print(os.getcwd())

d:\vscode_machineLearning\internship\Customer-Churn-Prediction\experiments\data_validation
d:\vscode_machineLearning\internship\Customer-Churn-Prediction


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv(r'artifacts/customer_churn_removed_col.csv')

## Entity

In [5]:
from pathlib import Path
from dataclasses import dataclass

In [6]:
@dataclass(frozen=True)
class DataValidationConfig:
    root_dir:Path
    data_dir:Path
    schema_check:dict
    make_data:dict

## configuration manager

## components

## pipeline